In [3]:
from main_head import *
# epsilon is the fixed parameter regarding optimizing CVaR, representing 1-\eta
epsilon = 0.05

# for the whole rolling approach, the number of days we consider in each period
## It is not just the physical days. Instead, in our paper, we consider the month as a unit. 
rolling_day = 1

# It denotes whether we include the shortsale constraints in our optimization model
## 0 represents that we incorporate the shortsale constraint: x_i >= 0 in our paper
shortsale_sign = 0

# cross validation choice
## cv_type == -1 means no  
## cv_type == 1 means gridsearch
cv_type = 1

# the K-fold number in cross validation
fold_number = 4

# the possible theta param corresponding to m in the paper) used in the cross validation
theta_param = [0.02, 0.04, 0.06, 0.08, 0.1]

# whether we take the risk-free rate into account in computing SR
## true means we need to minus the risk-free rate from the dataset and false means not
## Because DeMiguel's paper does not incorporate risk free rate, we do not incorporate that item too in the test statistics computation.
sharpe_ratio_open = False

#to make it more clearly, unit = 1 means the unit of portfolio dta is x%, unit = 100 means the unit of portfolio dta is x. 
unit = 1

#Data Set Choice Generator
def DTA_select(data_type):
    if data_type[-1] == '2':
        return [19970101, 201904, 20061231]
    else:
        if data_type in ['FF', 'IndustryPortfolios', 'MKT']:
            return [19630701, 200412, 19730631]
        elif data_type == 'S&P_sectors':
            return [19810101, 200212, 19901231]
        else:
            return [19700101, 200108, 19791231]

freq = "Monthly"
#Here as we all use the Monthly Data Set, we can fix that parameter
value = ""
#eq/not eq csv in the . Here we all use the original dataset class.

#dta type
## In DeMiguel's paper, the dta type are selected to be one in ['MKT', 'Country', 'S&P_sectors', 'IndustryPortfolios', 'FF']
## When his dataset extended to 2019 Q1, the dta type are selected to be one in ['MKT2', 'Country2', 'S&P_sectors2', 'IndustryPortfolios2', 'FF22']
## Their corresponding portfolio number is 3 (MKT), 9 (Country), 11 (S&P_sectors), 11 (IndustryPortfolios), 21 /24 for FF.
## And in our case study, dta type are selected to be 'IndustryIndices' (16)
data_config = ['S&P_sectors', 'IndustryPortfolios', 'Country', 'MKT', 'FF', 'FF']
data_config2 = ['S&P_sectors2', 'IndustryPortfolios2', 'Country2', 'MKT2', 'FF2', 'FF2']
port_nums = [11, 11, 9, 3, 21, 24]

MDD = []
for i in range(len(data_config)):
    data_type = data_config[i]
    portfolio_number = port_nums[i]

    #select part of the data in the .csv file.
    ## from start_time to train_test_split as the default train dataset; from train_test_split (not include) as the test dataset.
    [start_time, end_time, train_test_split] = DTA_select(data_type)

    cv_type = 1
    #data input
    Input_csv = Input(portfolio_number, freq, value, start_time, end_time, train_test_split, data_type)
    [data_head, data_parameter, csv_name] = Input_csv.parameter_output()
    [df_select, df_train] = Input_csv.data_load()

    df_factor = Input_csv.three_factor_load()
    #we do not include the risk free rate item into the computation of sharpe ratio
    if sharpe_ratio_open == False:
        rfr_data = 0
    else:
        rfr_data = Input_csv.risk_free_rate()

    cluster_num = 4
    df_state = pd.read_csv("../factor model/HMM_state.csv")
    str_state = [str(each_state) for each_state in list(df_state['Date'])]
    df_state['Date'] = str_state
    df_state_hmm = df_state[((df_state['Date'])>=str(start_time))&(df_state['Date']<str(end_time))]['state']
    ##hmm_type == -1 represents we use the pretrained result for our classification.
    hmm_type = -1

    # RSDR CVaR (HMM)
    method_name = "RSDR CVaR (HMM)"
    mean_constr = False
    fcvar_hmm_star = FCVaR_HMM_wasserstein(df_select, df_train, rolling_day, portfolio_number, df_factor, cluster_num, method_name, df_state_hmm, hmm_type, cv_type, mean_constr, ambiguity_param)
    fcvar_hmm_star.rolling(2)

    csv_name = f'../result_MDD/{portfolio_number}Monthly_{data_type}_v0319.csv'

    #initiation for the test metric module
    #print the output into a separate csv file, separated by the dataset type + header
    Output_csv = output(csv_name, data_head, data_parameter)
    Output_csv.head()
    #return statistics computation for our policy
    MDDV = Output_csv.return_info(fcvar_hmm_star, rfr_data)
    MDD.append(MDDV)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
done
For the model with strategy name  RSDR CVaR (HMM)
The Monthly Sharpe Ratio =  0.18365899130168772
CEQ =  0.006653691476498101
Drawdown =  -0.3194966654654253
Turnover =  0.06328870952599175
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
1

In [4]:
MDD

[-0.3194966654654253,
 -0.4459201787899765,
 -0.39065246304513923,
 -0.17114166491548344,
 -0.4349330269854237,
 -0.2729008333527273]